# Predict Target Structure

In [1]:
from scripts.protein_folding import *

## Download the ESMFold NIM



### Download NIM container

1.  Run

    ```bash
    docker pull nvcr.io/nvidia/nim/bionemo_esmfold_nim:24.03
    ```
2.  Download model

    ```bash
    mkdir -p ~/esmfold-nim/{weights,models}
    ```

    Then
    ```bash
    ngc registry model download-version nvidia/nim/bionemo-esmfold:protein-folding_noarchx1_bf16_24.03 --dest ~/esmfold-nim/models/
    ```

3.  Download the folding module weights.

    ```bash
    curl -L https://dl.fbaipublicfiles.com/fair-esm/models/esmfold_3B_v1.pt --output ~/esmfold-nim/weights/esmfold_3B_v1.pt
    ```

4.  Download the ESM2 3B language model weights.


    ```bash
    curl -L https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t36_3B_UR50D.pt --output ~/esmfold-nim/weights/esm2_t36_3B_UR50D.pt
    ```

5.  Download the contact regression model weights.


    ```bash
    curl -L https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t36_3B_UR50D-contact-regression.pt --output ~/esmfold-nim/weights/esm2_t36_3B_UR50D-contact-regression.pt
    ```

6.  Run NIM container

    ```bash
    docker run --rm -it \
    --name bionemo-esmfold \
    --gpus all \
    -v ~/esmfold-nim/models/bionemo-esmfold_vprotein-folding_noarchx1_bf16_24.03/:/config/models \
    -v ~/esmfold-nim/weights/:/esm_models \
    -e MODEL_PATH=/config/models \
    -p 8008:8008 \
    nvcr.io/nvidia/nim/bionemo_esmfold_nim:24.03
    ```
7.  Wait until it shows up like this

    ```bash
    I0421 21:04:17.461556 81 grpc_server.cc:2519] Started GRPCInferenceService at 0.0.0.0:8001
    I0421 21:04:17.461909 81 http_server.cc:4637] Started HTTPService at 0.0.0.0:8000
    I0421 21:04:17.503485 81 http_server.cc:320] Started Metrics Service at 0.0.0.0:8002
    ```


### Run health check

1.  Run health check at

    ```
    curl http://localhost:8008/health/ready
    ```

    Should return True
2.  To view docs and API schema, go to

    ```
    http://localhost:8008/docs
    ```




## Set up the urls

In [ ]:
import requests
import os

In [ ]:
base_url = "http://localhost:8008" 
query_url = base_url + "/protein-structure/esmfold/predict"
health_check_url = base_url + "/health/ready"

# output directory
output = "output/esmfold_result"
os.makedirs(output, exist_ok=True)


# run health check
response = requests.get(health_check_url)
assert response.text == "true", "Health check failed"

## Use your own PDB

## Check before you move on